In [2]:
import sys
import json
import numpy as np
import random
import time
import re
sys.path.append('../utils/')
from utils import gpt_decoder, load_json, load_jsonl
def write(fp, line):
    with open(fp, 'a', encoding='utf-8') as f:
        f.write(json.dumps(line) + '\n')

In [3]:
torque_datas = load_jsonl('./torque_timebench.jsonl')

In [6]:
torque_datas[0]

{'docid': 'docid_AFP_ENG_19970402.0459_sentid_1',
 'answer': ['said', 'expected', 'opened', 'rallied'],
 'passage': 'In Frankfurt, meanwhile, most economists said that the central council of the Bundesbank was expected to hold its key rates at a meeting on Thursday. The dollar opened weakly against the Japanese yen but then rallied.',
 'question': 'What events have already finished?'}

In [79]:
prompt = """You will be presented with a passage and a question about temporal ordering of event. 
You have to answer the temporal ordering question based on the passage.
The answer to the question comes from the trigger of events in the passage.
Passage: {}
Question: {}
Answer:
"""
prompt2 = """You will be presented with a passage and a question about temporal ordering of event.
You have to answer the question based on the passage.
The answer to the question is the trigger words of events in the passage.
Each question may have 0 or more answers.  If there is no answer that meets the requirements, output [unanswerable].
Only output the trigger word or [unanswerable].
Passage: {}
Question: {}
Answer:"""

prompt3 = """You will be presented with a passage and a question about temporal ordering of event.
You have to answer the question based on the passage.
Only output the event trigger words. 
If there is no answer, output [unanswerable]
Passage: {}
Question: {}
Answer:"""

prompt4 = """You will be presented with a passage and a question about temporal ordering of event.
You have to answer the question based on the passage.
If there is no answer, output [unanswerable]
Only output the event trigger words. 
Passage: {}
Question: {}
Answer:"""

prompt5 = """You will be presented with a passage and a question about temporal ordering of event.
You have to answer the question based on the passage.
Only output the event trigger words. 
Passage: {}
Question: {}
Answer:"""

prompt6 = """You will be presented with a passage and a question about temporal ordering of event.
You have to answer the question based on the passage.
Only output the event trigger words. 
Only output the event trigger words. 
Only output the event trigger words. 
Passage: {}
Question: {}
Answer:"""

prompt6 = """You will be presented with a passage and a question about temporal ordering of event.
You have to answer the question based on the passage.
Only output the event trigger word.
Only output the event trigger word. 
Only output the event trigger word. 
Passage: {}
Question: {}
Answer:"""

prompt6 = """You will be presented with a passage and a question about temporal ordering of event.
You have to answer the question based on the passage.
Only output the event trigger word.
Passage: {}
Question: {}
Answer:"""

In [164]:
prompt10 = """You will be presented with a passage and a question about temporal ordering of event.
You have to answer the question based on the passage.
Outputs all the correct event trigger.
Passage: {}
Question: {}
Answer:"""

prompt11 = """You will be presented with a passage and a question about temporal ordering of event.
You need to answer the question using the event trigger word, output all correct event trigger words(seperated with comma).
Passage: {}
Question: {}
Answer:"""

prompt12 = """You will be presented with a passage and a question about temporal ordering of event.
You need to answer the question base on the passage.
The answer format is single trigger word (e.g. left).
Passage: {}
Question: {}"""

prompt13 = """Get answers for the question based on the context.
An event trigger is a single word that represents the predicate of an event.
Answers are event triggers from the passage.
Output all correct event context.
Context: {}
Question: {}
Answer (Event Triggers):"""

In [166]:
N = 341
input = prompt13.format(torque_datas[N]['passage'], torque_datas[N]['question'])
result = gpt_decoder(input, n=10)
input, torque_datas[N]['answer'], result

("Get answers for the question based on the context.\nAn event trigger is a single word that represents the predicate of an event.\nAnswers are event triggers from the passage.\nOutput all correct event context.\nContext: Few thousand people defied heavy snow and gathered Saturday evening in Skopje's main square to join celebration and listened some of Macedonia's most popular music stars. The joyful street party ended in fireworks, but the celebration was expected to continue in restaurants and cafes that were exceptionally allowed to work overnight instead of having to close by midnight as usual.\nQuestion: What happened was expected to happen after the fireworks?\nAnswer (Event Triggers):",
 ['continue', 'allowed', 'work'],
 ['The celebration was expected to continue in restaurants and cafes.',
  'The celebration was expected to continue in restaurants and cafes.',
  'The celebration was expected to continue in restaurants and cafes.',
  'The celebration was expected to continue in 

In [65]:
def extract_answer(text):
    text = text.strip()
    triggers = text.split(',')
    triggers_satisfy = []
    for trigger in triggers:
        trigger = trigger.strip()
        trigger = trigger.split()
        if len(trigger) == 1:
            triggers_satisfy.append(trigger[0])
    if len(triggers_satisfy) > 0:
        return triggers_satisfy
    else:
        return None


In [136]:
print('1. The passage mentions that Wei had been under close watch at home.\n2. The passage also mentions that Wei was left briefly alone over the weekend.\n3. After being left alone, Wei chose that moment to hang himself in the bathroom of his apartment.\n\nTherefore, the events that occurred before Wei chose to hang himself are: \n1. Wei had been under close watch at home. \n2. Wei was left briefly alone over the weekend.')

1. The passage mentions that Wei had been under close watch at home.
2. The passage also mentions that Wei was left briefly alone over the weekend.
3. After being left alone, Wei chose that moment to hang himself in the bathroom of his apartment.

Therefore, the events that occurred before Wei chose to hang himself are: 
1. Wei had been under close watch at home. 
2. Wei was left briefly alone over the weekend.


In [97]:
def metric_cal(gold, pred):
    TP, FP, FN = 0, 0, 0
    for p in pred: #True positive
        if p in gold:
            TP += 1
    for p in pred: #False positive
        if p not in gold:
            FP += 1
    for g in gold: #False Negative
        if g not in pred:
            FN += 1
    P = TP / (TP + FP)
    R = TP / (TP + FN)
    if P + R == 0:
        F = 0
    else:
        F = 2 * P * R / (P + R)
    return P, R, F
        
    

In [98]:
metric_cal(['A', 'B', 'C'], ['D'])

(0.0, 0.0, 0)